In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))
import pandas as pd
from itertools import islice
import torch
from torch.utils.data import DataLoader
from utils.text_metrics import evaluate_all_metrics, save_metrics_to_json
from utils.temp_utils import *
from utils.gpt_models import DinoGPTCaptioner, DinoGPT2Captioner
from utils.chexpert_dataset import CheXpertDataset
from utils.padchest_dataset import PadChestGRDataset

# Data

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

CSV_PATH = "Datasets/CheXpertPlus/df_chexpert_plus_240401.csv"
IMG_ROOT = "Datasets/CheXpertPlus/PNG"

TEXT_COL = "section_impression"
PATH_COL = "path_to_image"

IMG_SIZE = 224
MAX_LEN = 64
NUM_BATCH = 8

tf = dino_image_transform(img_size=IMG_SIZE)

ds_train = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="train", transform=tf, text_col=TEXT_COL)
ds_valid = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="valid", transform=tf, text_col=TEXT_COL)
ds_test = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="test", transform=tf, text_col=TEXT_COL)

#labels = pd.read_csv(CSV_PATH)[TEXT_COL].tolist()

tokenizer = build_tokenizer_from_labels(captions=None)
pad_id = tokenizer.pad_token_id
eos_id = tokenizer.eos_token_id
bos_id = tokenizer.bos_token_id
collate_fn = CaptionCollate(tokenizer, pad_id)

train_loader = DataLoader(ds_train, batch_size=NUM_BATCH, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(ds_valid, batch_size=NUM_BATCH, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(ds_test, batch_size=NUM_BATCH, shuffle=False, collate_fn=collate_fn)

Using device: cuda
[INFO] Kept 47494/223462 rows with existing PNGs
[INFO] Kept 47494/223462 rows with existing PNGs
[INFO] Kept 47494/223462 rows with existing PNGs


In [3]:
tokenizer_size = tokenizer.vocab_size
print("Tokenizer size:", tokenizer_size)

Tokenizer size: 58996


# Model

In [4]:
# DINO ViT-S/16 hidden size is 384 
EMBEDDING_D_IMG = 384
N_PREFIX = (IMG_SIZE // 16) ** 2  # number of visual prefix tokens (including CLS)

def pick_heads(d_model, target_head_dim=64):
    h = max(1, round(d_model / target_head_dim))
    while d_model % h != 0: h -= 1
    return h

D_MODEL = 768
N_HEAD = pick_heads(D_MODEL, 64)  # -> 12


model = DinoGPTCaptioner(
    vocab_size=tokenizer.vocab_size,
    d_img=EMBEDDING_D_IMG,
    pad_id=pad_id,
    d_model=D_MODEL,
    n_layer=12,
    n_head=N_HEAD,
    n_prefix=N_PREFIX,           # number of visual prefix tokens
    max_seq_len=512,
    dino_model_id="facebook/dinov3-vits16-pretrain-lvd1689m",
    freeze_dino=True,
).to(device)

# Print model parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_params / 1_000_000:.2f} Millions")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable model parameters: {trainable_params / 1_000_000:.2f} Millions")

# Print model footprint
model_footprint_in_gb = (total_params * 4) * (1e-9)  # assuming 4 bytes per parameter (float32)
print(f"Approximate model footprint: {model_footprint_in_gb:.2f} GB")

# after model init
model.decoder.lm_head.weight = model.decoder.tok_emb.weight  # weight tying

Total model parameters: 198.08 Millions
Trainable model parameters: 176.48 Millions
Approximate model footprint: 0.79 GB


# Train Parameters

In [5]:
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=3e-4, weight_decay=1e-2
)
loss = sequence_ce_loss
NUM_EPOCHS = 100
BATCHES_PER_EPOCH = 10

# Training

In [6]:
import time
time_start = time.time()
for epoch in range(NUM_EPOCHS):
    slice_train_loader = islice(train_loader, BATCHES_PER_EPOCH)
    slice_valid_loader = islice(valid_loader, BATCHES_PER_EPOCH)
    train_stats = train_one_epoch(model, slice_train_loader, optimizer, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss, grad_clip=1.0)
    val_stats = evaluate(model, slice_valid_loader, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss)
    print(f"Epoch {epoch + 1}: Train Loss={train_stats['loss']:.4f}, PPL={train_stats['ppl']:.2f} | "
            f"Val Loss={val_stats['val_loss']:.4f}, Val PPL={val_stats['val_ppl']:.2f}")
training_time = time.time() - time_start

Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.29it/s]


Epoch 1: Train Loss=9.5661, PPL=19159.79 | Val Loss=8.3382, Val PPL=4230.72


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.31it/s]


Epoch 2: Train Loss=7.8289, PPL=2561.26 | Val Loss=7.5083, Val PPL=1862.58


Evaluating: 100%|██████████| 10/10 [00:05<00:00,  1.94it/s]


...

Epoch 98: Train Loss=4.6557, PPL=107.24 | Val Loss=4.5327, Val PPL=96.76


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.23it/s]


Epoch 99: Train Loss=4.7361, PPL=117.86 | Val Loss=4.5205, Val PPL=95.47


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.27it/s]

Epoch 100: Train Loss=4.6628, PPL=109.61 | Val Loss=4.5331, Val PPL=96.71


# Test Parameters

In [7]:
BATCHES_PER_TEST = 1
GREEDY_DECODE = True
TEST_MAX_LEN = 256
TEST_TOP_P = 0.9
TEST_TEMPERATURE = 0.9

# Test

In [8]:
slice_test_loader = islice(test_loader, BATCHES_PER_TEST)
test_stats = evaluate(model, slice_test_loader, device, pad_id, num_batches=BATCHES_PER_TEST)
print(f"Test Loss={test_stats['val_loss']:.4f}, Test PPL={test_stats['val_ppl']:.2f}")

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  1.97it/s]

Test Loss=4.4065, Test PPL=81.98


# Test Report Generation

In [9]:
# capitalize first word and first word after each "."
def capitalize_sentences(s):
    parts = s.split('. ')
    parts = [p[:1].upper() + p[1:] if p else '' for p in parts]
    return '. '.join(parts)

generated_text = []
target_text = []
iteration = 0

with torch.no_grad():
    for pixel_values, ids_loader, paths, raw_labels in test_loader:
        iteration += 1
        pixel_values = pixel_values.to(device)
        # gen_ids = model.generate(
        #     pixel_values=pixel_values,
        #     bos_id=bos_id,
        #     eos_id=eos_id,
        #     max_new_tokens=TEST_MAX_LEN,
        #     beam_size=3,                # Set your desired beam size
        #     temperature=TEST_TEMPERATURE
        # )

        info = model.generate_with_logging(
            pixel_values=pixel_values,          # [B, C, H, W]
            bos_id=tokenizer.bos_token_id,
            eos_id=tokenizer.eos_token_id,
            tokenizer=tokenizer,
            preset="safe_sample",
            stop_sequences=["\n\n", "Impression:"],
            max_new_tokens=256,
        )

        # print("batch sequences shape:", info["sequences"].shape)
        # for i, s in enumerate(info["per_sample"]):
        #     print(f"[sample {i}] hit_eos={s['stopping']['hit_eos']} repetition={s['repetition']}")
        #     if "generated" in s["text"]:
        #         print(capitalize_sentences(s["text"]["generated"]))
        #         print("[Target text]", capitalize_sentences(raw_labels[i]))

        generated_text.extend([s["text"]["generated"] for s in info["per_sample"]])
        target_text.extend(raw_labels)

        # eval_results = evaluate_all_metrics(raw_labels, [s["text"]["generated"] for s in info["per_sample"]], evaluation_mode="CheXagent")
        # for metric, scores in eval_results.items():
        #     print(f"{metric}: {scores}")

        # save_metrics_to_json(eval_results, f"lstm-vs-gpt/results/gpt_model_results_{NUM_EPOCHS}_Chexpert.json")


        # print("Predictions (first batch):")
        # for i in range(gen_ids.size(0)):
        #     text_gen = tokenizer.decode(gen_ids[i].tolist())
        #     text_tgt = tokenizer.decode(ids_loader[i].tolist())
        #     print(f"\nGEN {i+1}:", capitalize_sentences(text_gen))
        #     print(f"TGT {i+1}:", capitalize_sentences(text_tgt))
        #     results = evaluate_all_metrics([text_tgt], [text_gen], evaluation_mode="CheXagent")
        #     for metric, scores in results.items():
        #         print(f"{metric}: {scores}")
        del pixel_values, ids_loader, paths, raw_labels, info
        torch.cuda.empty_cache()
        # break
        if iteration >= 10:  # Limit to 10 iterations for testing
            break

eval_results = evaluate_all_metrics(generated_text, target_text, evaluation_mode="CheXagent")
for metric, scores in eval_results.items():
    print(f"{metric}: {scores}")
eval_results["training_time_seconds"] = training_time
save_metrics_to_json(eval_results, f"lstm-vs-gpt/results/gpt_model_results_{NUM_EPOCHS}_Chexpert.json")

Using device: cuda:0
chexbert_f1_weighted: 0.36491390171440097
chexbert_f1_micro: 0.3927855711422846
chexbert_f1_macro: 0.20486577822308455
chexbert_f1_micro_5: 0.35125448028673834
chexbert_f1_macro_5: 0.3219951063429324
radgraph_f1_RG_E: 0.11892152991772556
radgraph_f1_RG_ER: 0.10539613373027526


In [10]:
import re
import string

def clean_text(text: str) -> str:
    # lowercase
    text = text.lower()

    # remove enumerators like "1." or "23." but KEEP decimals like "2.5"
    text = re.sub(r'(?<!\d)\b\d+\.(?!\d)', ' ', text)

    # remove all punctuation EXCEPT "."
    punctuation = string.punctuation.replace('.', '')
    text = text.translate(str.maketrans('', '', punctuation))

    # normalize spaces around periods to ". " → ". "
    text = re.sub(r'\s*\.\s*', '. ', text)

    # collapse multiple spaces and trim
    text = re.sub(r'\s+', ' ', text).strip()

    # capitalize first word and first word after each "."
    def capitalize_sentences(s):
        parts = s.split('. ')
        parts = [p[:1].upper() + p[1:] if p else '' for p in parts]
        return '. '.join(parts)
    text = capitalize_sentences(text)

    return text

# Example
text = "1.  STABLE SMALL LEFT INTERNAL JUGULAR OPACITIES... 2.5 cm nodule; item 2. next. 3. Done."
print(clean_text(text))


# Example
text = """
 1.  INTERVAL PLACEMENT OF A RIGHT INTERNAL JUGULAR VENOUS SHEATH 
WITH THE DISTAL TIP IN THE PROXIMAL SUPERIOR VENA CAVA.  NO 
PNEUMOTHORAX.
 
 2.  STABLE POSITION OF NASOGASTRIC TUBE, FEEDING TUBE, TRACHEOSTOMY 
CANULA, LEFT INTERNAL JUGULAR CENTRAL VENOUS CATHETER, AND LEFT UPPER 
EXTREMITY PICC.  
 
 3.  NO SIGNIFICANT INTERVAL CHANGE IN HYPEREXPANDED LUNG VOLUMES, 
RIGHT BASILAR OPACITIES, SMALL BILATERAL PLEURAL EFFUSIONS, TENTING 
OF THE RIGHT HEMIDIAPHRAGM AND BIAPICAL PLEURAL THICKENING. 
 
 """
cleaned_text = clean_text(text)
print(cleaned_text)


Stable small left internal jugular opacities. . . 2. 5 cm nodule item next. Done.
Interval placement of a right internal jugular venous sheath with the distal tip in the proximal superior vena cava. No pneumothorax. Stable position of nasogastric tube feeding tube tracheostomy canula left internal jugular central venous catheter and left upper extremity picc. No significant interval change in hyperexpanded lung volumes right basilar opacities small bilateral pleural effusions tenting of the right hemidiaphragm and biapical pleural thickening.


In [11]:
text = "1.  STABLE SMALL LEFT INTERNAL JUGULAR OPACITIES WITH PATCHY TUBE AND NASOGASTRIC TUBES, RIGHT LOWER MEDIASTINAL SIDED CATHETER.  NO SIGNIFICANT CHANGE IN THE PREVIOUS STUDYDEMONSTRATE ATELECTASIS O"
print("Original text:", text)
encoded = tokenizer.encode(text)
words = text.split()
print("Number of words:", len(words), "Number of tokens:", len(encoded), "pad_id:", pad_id, "eos_id:", eos_id, "bos_id:", bos_id)
print("BOS token id:", tokenizer.bos_token_id, "EOS token id:", tokenizer.eos_token_id, "PAD token id:", tokenizer.pad_token_id)
print(encoded)
for token_id in encoded:    
    print(f"Token ID: {token_id}, Token: {tokenizer.decode([token_id])}")

print("\nAfter lowercasing:")
textlower = cleaned_text
encoded = tokenizer.encode(textlower)
words = textlower.split()
print("Number of words:", len(words), "Number of tokens:", len(encoded), "pad_id:", pad_id, "eos_id:", eos_id, "bos_id:", bos_id)
print("BOS token id:", tokenizer.bos_token_id, "EOS token id:", tokenizer.eos_token_id, "PAD token id:", tokenizer.pad_token_id)
print(encoded)
for token_id in encoded:    
    print(f"Token ID: {token_id}, Token: {tokenizer.decode([token_id])}")

Original text: 1.  STABLE SMALL LEFT INTERNAL JUGULAR OPACITIES WITH PATCHY TUBE AND NASOGASTRIC TUBES, RIGHT LOWER MEDIASTINAL SIDED CATHETER.  NO SIGNIFICANT CHANGE IN THE PREVIOUS STUDYDEMONSTRATE ATELECTASIS O
Number of words: 27 Number of tokens: 43 pad_id: 0 eos_id: 102 bos_id: 101
BOS token id: 101 EOS token id: 102 PAD token id: 0
[101, 122, 119, 6111, 1353, 1286, 4422, 34986, 5552, 39280, 49176, 1114, 10085, 1183, 7159, 1105, 9468, 7301, 32519, 11182, 117, 1268, 2211, 2394, 34979, 7050, 11641, 5855, 30682, 119, 1185, 2418, 1849, 1107, 1103, 2166, 2025, 31386, 8756, 18465, 14229, 184, 102]
Token ID: 101, Token: 
Token ID: 122, Token: 1
Token ID: 119, Token: .
Token ID: 6111, Token: stable
Token ID: 1353, Token: small
Token ID: 1286, Token: left
Token ID: 4422, Token: internal
Token ID: 34986, Token: jug
Token ID: 5552, Token: ##ular
Token ID: 39280, Token: opa
Token ID: 49176, Token: ##cities
Token ID: 1114, Token: with
Token ID: 10085, Token: patch
Token ID: 1183, Token: ##y
T